In [ ]:
!pip install duckdb duckdb-engine llama-index

In [1]:
import yaml, os, openai
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
from llama_index import SQLDatabase, SimpleDirectoryReader, WikipediaReader, Document, ServiceContext, set_global_service_context
from llama_index.indices.struct_store import (
                                            NLSQLTableQueryEngine,
                                            SQLTableRetrieverQueryEngine,
                                            )

from sqlalchemy import (
                        create_engine,
                        MetaData,
                        Table,
                        Column,
                        String,
                        Integer,
                        select,
                        column,
                        insert
                        )

In [2]:
# creates a SQLAlchemy engine object that connects to an in-memory DuckDB database.
engine = create_engine("duckdb:///:memory:") # https://duckdb.org/
metadata_obj = MetaData()

In [3]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [4]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

In [5]:
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States"},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)
     

In [6]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [7]:
sql_database = SQLDatabase(engine, include_tables=['city_stats'])

/Users/1zuu/anaconda3/envs/llm/lib/python3.10/site-packages/duckdb_engine/__init__.py:162: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


## Query Index

    - An Index is a data structure that allows us to quickly retrieve relevant context for a user query
    - We are going to use the NLSQLTableQueryEngine as an query engine and run queries against it.

In [8]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['ENGINE'] = credentials['ENGINE']

openai.api_key = credentials['OPENAI_API_KEY']
openai.api_base = credentials['OPENAI_API_BASE']
openai.api_type = credentials['OPENAI_API_TYPE']
openai.api_version = credentials['OPENAI_API_VERSION']
openai.engine = credentials['ENGINE']

In [9]:
llm = OpenAI(
            openai_api_key=os.environ["OPENAI_API_KEY"],
            engine = os.environ["ENGINE"],
            model='gpt-3.5-turbo',
            temperature=0.0, 
            max_tokens = 256
            )

In [10]:
service_context = ServiceContext.from_defaults(llm=llm)

In [11]:
#query_engine_openai = NLSQLTableQueryEngine(sql_database)
query_engine_openai = NLSQLTableQueryEngine(sql_database, service_context=service_context)

In [12]:
response = query_engine_openai.query("Which city has the highest population?")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: f79aa605********************4638. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
response.response